In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, DateType

# 创建 SparkSession
spark = SparkSession.builder \
    .appName("ProductTableLoader") \
    .getOrCreate()


In [0]:
# 定义产品表的表结构
product_schema = StructType([
    StructField("Product ID", StringType(), True),
    StructField("Product Name", StringType(), True),
    StructField("Category", StringType(), True),
    StructField("Brand", StringType(), True),
    StructField("Description", StringType(), True),
    StructField("Cost Price", FloatType(), True),
    StructField("Selling Price", FloatType(), True),
    StructField("Supplier ID", StringType(), True),
    StructField("Stock Keeping Unit (SKU)", StringType(), True)
])

In [0]:
# 从字符串中读取产品记录并加载到 DataFrame 中
product_records = [
    ("P1","Product1","Category1","Brand1","Description1",10.5,20.5,"Supplier1","SKU1"),
    ("P2","Product2","Category2","Brand2","Description2",15.0,25.0,"Supplier2","SKU2"),
    ("P3","Product3","Category3","Brand3","Description3",12.0,22.0,"Supplier3","SKU3")
]

product_df = spark.createDataFrame(product_records, schema=product_schema)

# 加载到 ODS 表中
# 假设 ODS 表已存在或需要创建
ods_table_name = "exp1_ods_product_table"

# 将时间 DataFrame 保存到 ODS 表中
product_df.write.mode("overwrite").format("parquet").saveAsTable(ods_table_name)

# 从 ODS 表中读取数据
product_ods_df = spark.table(ods_table_name)

# 展示 DataFrame
product_ods_df.show()

+----------+------------+---------+------+------------+----------+-------------+-----------+------------------------+
|Product ID|Product Name| Category| Brand| Description|Cost Price|Selling Price|Supplier ID|Stock Keeping Unit (SKU)|
+----------+------------+---------+------+------------+----------+-------------+-----------+------------------------+
|        P1|    Product1|Category1|Brand1|Description1|      10.5|         20.5|  Supplier1|                    SKU1|
|        P2|    Product2|Category2|Brand2|Description2|      15.0|         25.0|  Supplier2|                    SKU2|
|        P3|    Product3|Category3|Brand3|Description3|      12.0|         22.0|  Supplier3|                    SKU3|
+----------+------------+---------+------+------------+----------+-------------+-----------+------------------------+



In [0]:
# product的dimension表
from pyspark.sql import SparkSession
from pyspark.sql.types import *

# 创建 SparkSession
spark = SparkSession.builder \
    .appName("Product Dimension Table Creation") \
    .getOrCreate()

product_ods_df = spark.table(ods_table_name)

# 创建维度表
dimension_df = product_ods_df.select("Product ID", "Product Name", "Category", "Brand", "Description", "Cost Price", "Selling Price", "Supplier ID","Stock Keeping Unit (SKU)")

# 加载到 DIM 表中
# 假设 DIM 表已存在或需要创建
dim_table_name = "exp1_dim_product_table"

# 将时间 DataFrame 保存到 DIM 表中
dimension_df.write.mode("overwrite").format("parquet").saveAsTable(dim_table_name)

dimension_df.show()


+----------+------------+---------+------+------------+----------+-------------+-----------+------------------------+
|Product ID|Product Name| Category| Brand| Description|Cost Price|Selling Price|Supplier ID|Stock Keeping Unit (SKU)|
+----------+------------+---------+------+------------+----------+-------------+-----------+------------------------+
|        P1|    Product1|Category1|Brand1|Description1|      10.5|         20.5|  Supplier1|                    SKU1|
|        P2|    Product2|Category2|Brand2|Description2|      15.0|         25.0|  Supplier2|                    SKU2|
|        P3|    Product3|Category3|Brand3|Description3|      12.0|         22.0|  Supplier3|                    SKU3|
+----------+------------+---------+------+------------+----------+-------------+-----------+------------------------+



In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
from pyspark.sql.functions import to_date

# Define the inventory table schema
inventory_schema = StructType([
    StructField("Product ID", StringType(), True),
    StructField("Warehouse ID", StringType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("Stock Date", StringType(), True)
])

# Construct the test data
warehouse_data = [
    ("P1", "W1", 100, "2023-01-01"),
    ("P2", "W2", 150, "2023-01-02"),
    ("P3", "W1", 200, "2023-01-03")
]

# Create DataFrame
inventory_df = spark.createDataFrame(warehouse_data, schema=inventory_schema)

# Apply the to_date() transformation to the "Stock Date" column
inventory_df = inventory_df.withColumn("Stock Date", to_date(inventory_df["Stock Date"]))

# 加载到 OD 表中
# 假设 OD 表已存在或需要创建
ods_table_name = "exp1_ods_inventory_table"

# 将时间 DataFrame 保存到 OD 表中
inventory_df.write.mode("overwrite").format("parquet").saveAsTable(ods_table_name)

# 从 OD 表中读取数据
inventory_ods_df = spark.table(ods_table_name)

# 展示 DataFrame
inventory_ods_df.show()

+----------+------------+--------+----------+
|Product ID|Warehouse ID|Quantity|Stock Date|
+----------+------------+--------+----------+
|        P2|          W2|     150|2023-01-02|
|        P1|          W1|     100|2023-01-01|
|        P3|          W1|     200|2023-01-03|
+----------+------------+--------+----------+



In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType
from pyspark.sql.functions import to_date
from datetime import datetime

# Define sales table structure
sales_schema = StructType([
    StructField("Order ID", StringType(), True),
    StructField("Product ID", StringType(), True),
    StructField("Customer ID", StringType(), True),
    StructField("Quantity", IntegerType(), True),
    StructField("Sales Date", DateType(), True),
    StructField("Unit Price", DoubleType(), True),
    StructField("Total Price", DoubleType(), True),
    StructField("Payment Method", StringType(), True)
])

# Construct test data with updated date format
sales_data = [
    ("ORD001", "P1", "CUST001", 2, datetime.strptime("2023-01-01", "%Y-%m-%d"), 10.0, 20.0, "Cash"),
    ("ORD002", "P2", "CUST002", 3, datetime.strptime("2023-01-02", "%Y-%m-%d"), 15.0, 45.0, "Credit Card"),
    ("ORD003", "P3", "CUST003", 1, datetime.strptime("2023-01-03", "%Y-%m-%d"), 20.0, 20.0, "Online Payment")
]

# Create DataFrame
sales_df = spark.createDataFrame(sales_data, schema=sales_schema)

# Show DataFrame
print("Mock Sales Data:")
sales_df.show()

# 加载到 OD 表中
# 假设 OD 表已存在或需要创建
ods_table_name = "exp1_ods_sales_table"

# 将时间 DataFrame 保存到 OD 表中
sales_df.write.mode("overwrite").format("parquet").saveAsTable(ods_table_name)

# 从 OD 表中读取数据
sales_ods_df = spark.table(ods_table_name)

# 展示 DataFrame
sales_ods_df.show()

Mock Sales Data:
+--------+----------+-----------+--------+----------+----------+-----------+--------------+
|Order ID|Product ID|Customer ID|Quantity|Sales Date|Unit Price|Total Price|Payment Method|
+--------+----------+-----------+--------+----------+----------+-----------+--------------+
|  ORD001|        P1|    CUST001|       2|2023-01-01|      10.0|       20.0|          Cash|
|  ORD002|        P2|    CUST002|       3|2023-01-02|      15.0|       45.0|   Credit Card|
|  ORD003|        P3|    CUST003|       1|2023-01-03|      20.0|       20.0|Online Payment|
+--------+----------+-----------+--------+----------+----------+-----------+--------------+

+--------+----------+-----------+--------+----------+----------+-----------+--------------+
|Order ID|Product ID|Customer ID|Quantity|Sales Date|Unit Price|Total Price|Payment Method|
+--------+----------+-----------+--------+----------+----------+-----------+--------------+
|  ORD003|        P3|    CUST003|       1|2023-01-03|      20.

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# 定义时间表的表结构
time_schema = StructType([
    StructField("Date", StringType(), nullable=False),
    StructField("Day", StringType(), nullable=False),
    StructField("Month", StringType(), nullable=False),
    StructField("Quarter", StringType(), nullable=False),
    StructField("Year", IntegerType(), nullable=False)
])

# 构造测试数据
test_data = [
    ("2023-01-01", "Monday", "January", "Q1", 2023),
    ("2023-01-02", "Tuesday", "January", "Q1", 2023),
    ("2023-01-03", "Wednesday", "January", "Q1", 2023),
    # 添加更多测试数据...
]

# 创建 DataFrame
time_df = spark.createDataFrame(test_data, schema=time_schema)

# 加载到 ODS 表中
# 假设 ODS 表已存在或需要创建
ods_table_name = "exp1_ods_time_table"

# 将时间 DataFrame 保存到 OD 表中
time_df.write.mode("overwrite").format("parquet").saveAsTable(ods_table_name)

# 从 OD 表中读取数据
ods_time_df = spark.table(ods_table_name)

# 显示 OD 表数据
ods_time_df.show()


+----------+---------+-------+-------+----+
|      Date|      Day|  Month|Quarter|Year|
+----------+---------+-------+-------+----+
|2023-01-03|Wednesday|January|     Q1|2023|
|2023-01-02|  Tuesday|January|     Q1|2023|
|2023-01-01|   Monday|January|     Q1|2023|
+----------+---------+-------+-------+----+



In [0]:
# date 的dim表
# 选择维度列
ods_time_df = spark.table("exp1_ods_time_table")

# 选择维度列
dimension_date_df = ods_time_df.select("Date", "Day", "Month", "Quarter", "Year")

# 如果有必要，添加额外的维度列
# dimension_df = dimension_df.withColumn("SomeOtherDimension", some_function())

dim_table_name = "exp1_dim_date_table"

# 写入维度表
dimension_date_df.write.mode("overwrite").saveAsTable(dim_table_name)

dimension_date_df.show()

+----------+---------+-------+-------+----+
|      Date|      Day|  Month|Quarter|Year|
+----------+---------+-------+-------+----+
|2023-01-03|Wednesday|January|     Q1|2023|
|2023-01-02|  Tuesday|January|     Q1|2023|
|2023-01-01|   Monday|January|     Q1|2023|
+----------+---------+-------+-------+----+



In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

# 定义供应商表结构
supplier_schema = StructType([
    StructField("SupplierID", StringType(), True),
    StructField("SupplierName", StringType(), True),
    StructField("ContactPerson", StringType(), True),
    StructField("Address", StringType(), True),
    StructField("Phone", StringType(), True),
    StructField("Email", StringType(), True)
])

# 构造测试数据
supplier_data = [
    ("S1", "Supplier A", "John Doe", "123 Main St, City, Country", "123-456-7890", "supplierA@example.com"),
    ("S2", "Supplier B", "Jane Smith", "456 Oak St, City, Country", "456-789-0123", "supplierB@example.com")
]

# 创建供应商 DataFrame
supplier_df = spark.createDataFrame(supplier_data, schema=supplier_schema)

# 加载到 ODS 表中
# 假设 ODS 表已存在或需要创建
ods_table_name = "exp1_ods_supplier_table"

# 将时间 DataFrame 保存到 ODS 表中
supplier_df.write.mode("overwrite").format("parquet").saveAsTable(ods_table_name)

# 从 ODS 表中读取数据
suppliers_ods_df = spark.table(ods_table_name)

# 展示 DataFrame
suppliers_ods_df.show()


+----------+------------+-------------+--------------------+------------+--------------------+
|SupplierID|SupplierName|ContactPerson|             Address|       Phone|               Email|
+----------+------------+-------------+--------------------+------------+--------------------+
|        S2|  Supplier B|   Jane Smith|456 Oak St, City,...|456-789-0123|supplierB@example...|
|        S1|  Supplier A|     John Doe|123 Main St, City...|123-456-7890|supplierA@example...|
+----------+------------+-------------+--------------------+------------+--------------------+



In [0]:
# 选择维度列
ods_supplier_df = spark.table("exp1_ods_supplier_table")

# 选择维度列
dimension_supplier_df = ods_supplier_df.select("SupplierID", "SupplierName", "ContactPerson", "Address", "Phone", "Email")

dim_table_name = "exp1_dim_supplier_table"

# 写入维度表
dimension_supplier_df.write.mode("overwrite").saveAsTable(dim_table_name)

dimension_supplier_df.show()

+----------+------------+-------------+--------------------+------------+--------------------+
|SupplierID|SupplierName|ContactPerson|             Address|       Phone|               Email|
+----------+------------+-------------+--------------------+------------+--------------------+
|        S2|  Supplier B|   Jane Smith|456 Oak St, City,...|456-789-0123|supplierB@example...|
|        S1|  Supplier A|     John Doe|123 Main St, City...|123-456-7890|supplierA@example...|
+----------+------------+-------------+--------------------+------------+--------------------+



In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# 定义仓库表的结构
warehouse_schema = StructType([
    StructField("WarehouseID", StringType(), True),
    StructField("WarehouseName", StringType(), True),
    StructField("Location", StringType(), True),
    StructField("Capacity", IntegerType(), True)
])

# 构造测试数据
test_data = [
    ("W1", "Warehouse A", "New York", 1000),
    ("W2", "Warehouse B", "Los Angeles", 1500),
    ("W3", "Warehouse C", "Chicago", 1200)
]

# 将测试数据转换为 DataFrame
test_df = spark.createDataFrame(test_data, schema=warehouse_schema)

# 加载到 OD 表中
# 假设 OD 表已存在或需要创建
ods_table_name = "exp1_ods_warehouse_table"

# 将时间 DataFrame 保存到 OD 表中
test_df.write.mode("overwrite").format("parquet").saveAsTable(ods_table_name)

# 从 OD 表中读取数据
warehouse_ods_df = spark.table(ods_table_name)

# 展示 DataFrame
warehouse_ods_df.show()


+-----------+-------------+-----------+--------+
|WarehouseID|WarehouseName|   Location|Capacity|
+-----------+-------------+-----------+--------+
|         W2|  Warehouse B|Los Angeles|    1500|
|         W1|  Warehouse A|   New York|    1000|
|         W3|  Warehouse C|    Chicago|    1200|
+-----------+-------------+-----------+--------+



In [0]:
# 选择维度列
ods_warehouse_df = spark.table("exp1_ods_warehouse_table")

# 选择维度列
dimension_warehouse_df = ods_warehouse_df.select("WarehouseID", "WarehouseName", "Location", "Capacity")

dim_table_name = "exp1_dim_warehouse_table"

# 写入维度表
dimension_warehouse_df.write.mode("overwrite").saveAsTable(dim_table_name)

dimension_warehouse_df.show()

+-----------+-------------+-----------+--------+
|WarehouseID|WarehouseName|   Location|Capacity|
+-----------+-------------+-----------+--------+
|         W2|  Warehouse B|Los Angeles|    1500|
|         W1|  Warehouse A|   New York|    1000|
|         W3|  Warehouse C|    Chicago|    1200|
+-----------+-------------+-----------+--------+



In [0]:
# 选择维度列
ods_inventory_df = spark.table("exp1_ods_inventory_table")
dim_product_df = spark.table("exp1_dim_product_table")
dim_date_df = spark.table("exp1_dim_date_table")
dim_warehouse_df = spark.table("exp1_dim_warehouse_table")

# 选择dwd表的列
dwd_inventory_df = ods_inventory_df.select("Product ID", "Warehouse ID", "Quantity", "Stock Date")

dwd_inventory_df = dwd_inventory_df\
                    .join(dim_product_df, dwd_inventory_df['Product ID'] == dim_product_df['Product ID'],"left")\
                    .select(dwd_inventory_df["Product ID"],
                            dwd_inventory_df["Warehouse ID"],
                            dwd_inventory_df["Quantity"],
                            dwd_inventory_df["Stock Date"],
                            dim_product_df['Product Name'],
                            dim_product_df['Category'],
                            dim_product_df['Brand'])
dwd_inventory_df = dwd_inventory_df\
                    .join(dim_date_df, dwd_inventory_df['Stock Date'] == dim_date_df['Date'],"left")\
                    .select(dwd_inventory_df["Product ID"],
                            dwd_inventory_df["Warehouse ID"],
                            dwd_inventory_df["Quantity"],
                            dwd_inventory_df["Stock Date"],
                            dwd_inventory_df['Product Name'],
                            dwd_inventory_df['Category'],
                            dwd_inventory_df['Brand'],
                            dim_date_df['Quarter'].alias("Stock Quarter"))
dwd_inventory_df = dwd_inventory_df\
                    .join(dim_warehouse_df, dwd_inventory_df['Warehouse ID'] == dim_warehouse_df['WarehouseID'],"left")\
                    .select(dwd_inventory_df["Product ID"].alias("ProductID"),
                            dwd_inventory_df["Warehouse ID"].alias("WarehouseID"),
                            dwd_inventory_df["Quantity"],
                            dwd_inventory_df["Stock Date"].alias("StockDate"),
                            dwd_inventory_df['Product Name'].alias("ProductName"),
                            dwd_inventory_df['Category'],
                            dwd_inventory_df['Brand'],
                            dwd_inventory_df['Stock Quarter'].alias("StockQuarter"),
                            dim_warehouse_df['WarehouseName'],
                            dim_warehouse_df['Location'].alias("WarehouseLocation"))\

dwd_table_name = "exp1_dwd_inventory_table"

# 写入维度表
dwd_inventory_df.write.mode("overwrite").saveAsTable(dwd_table_name)

dwd_inventory_df.show()

+---------+-----------+--------+----------+-----------+---------+------+------------+-------------+-----------------+
|ProductID|WarehouseID|Quantity| StockDate|ProductName| Category| Brand|StockQuarter|WarehouseName|WarehouseLocation|
+---------+-----------+--------+----------+-----------+---------+------+------------+-------------+-----------------+
|       P2|         W2|     150|2023-01-02|   Product2|Category2|Brand2|          Q1|  Warehouse B|      Los Angeles|
|       P1|         W1|     100|2023-01-01|   Product1|Category1|Brand1|          Q1|  Warehouse A|         New York|
|       P3|         W1|     200|2023-01-03|   Product3|Category3|Brand3|          Q1|  Warehouse A|         New York|
+---------+-----------+--------+----------+-----------+---------+------+------------+-------------+-----------------+



In [0]:
# 选择维度列
ods_sales_df = spark.table("exp1_ods_sales_table")
dim_product_df = spark.table("exp1_dim_product_table")

# 选择dwd表的列
dwd_sales_df = ods_sales_df.select("Order ID", "Product ID", "Customer ID", "Quantity","Sales Date","Unit Price","Total Price","Payment Method")

dwd_sales_df = dwd_sales_df\
                    .join(dim_product_df, dwd_sales_df['Product ID'] == dim_product_df['Product ID'],"left")\
                    .select(dwd_sales_df["Order ID"].alias("OrderID"),
                            dwd_sales_df["Product ID"].alias("ProductID"),
                            dwd_sales_df["Customer ID"].alias("CustomerID"),
                            dwd_sales_df["Quantity"],
                            dwd_sales_df["Sales Date"].alias("SalesDate"),
                            dwd_sales_df['Unit Price'].alias("UnitPrice"),
                            dwd_sales_df['Total Price'].alias("TotalPrice"),
                            dwd_sales_df['Payment Method'].alias("PaymentMethod"),
                            dim_product_df['Product Name'].alias("ProductName"),
                            dim_product_df['Category'],
                            dim_product_df['Brand'])

dwd_table_name = "exp1_dwd_sales_table"
# 写入维度表
dwd_sales_df.write.mode("overwrite").saveAsTable(dwd_table_name)

dwd_sales_df.show()

+-------+---------+----------+--------+----------+---------+----------+--------------+-----------+---------+------+
|OrderID|ProductID|CustomerID|Quantity| SalesDate|UnitPrice|TotalPrice| PaymentMethod|ProductName| Category| Brand|
+-------+---------+----------+--------+----------+---------+----------+--------------+-----------+---------+------+
| ORD003|       P3|   CUST003|       1|2023-01-03|     20.0|      20.0|Online Payment|   Product3|Category3|Brand3|
| ORD002|       P2|   CUST002|       3|2023-01-02|     15.0|      45.0|   Credit Card|   Product2|Category2|Brand2|
| ORD001|       P1|   CUST001|       2|2023-01-01|     10.0|      20.0|          Cash|   Product1|Category1|Brand1|
+-------+---------+----------+--------+----------+---------+----------+--------------+-----------+---------+------+



In [0]:
# 停止 SparkSession
spark.stop()